In [8]:
import os, re, sys
from datetime import date
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser as utp
import scripts.tools.redmine_cache as rmc
from scripts.software_frs import frs_traceability2 as FRS

def reload():
    global iie, utp, rmc, UserTestsParser, RequirementExtracter
    
    iie = importlib.reload(iie)
    utp = importlib.reload(utp)
    rmc = importlib.reload(rmc)
    UserTestsParser = utp.UserTestsParser
    RequirementExtracter = iie.RequirementExtracter

reload()

In [9]:
_client = None

def _get_client():
    global _client
    _client = _client or rmc.RedmineClient()
    return _client

def load_issues():
    return _get_client().get_filtered([('project.identifier', '==', 'pbssoftware')])

In [10]:
rtags = [
    "URS",
    "FRS",
    "SWDS",
    "CS",
    "BUG"
]

ttags = [
    "USR"
]

_ignore = {3194, 3287}
_sprints = {
    'Legacy',
    '3.0',
    '3.0.1',
    '3.0.2',
    '3.0.3',
    '3.0.4',
    '3.0.5',
    '3.0.6',
    '3.0.7'
}

def relevant(i):
    return i.sprint_milestone.name in _sprints and i.id not in _ignore and i.status != "Rejected"

def relevent38(i):
    return i.sprint_milestone.name == "3.0.8" and i.id not in _ignore and i.status != "Rejected"

In [11]:
issues = load_issues()
issues2 = list(filter(relevant, issues.values()))

parser = iie.IssuetrackerParser(rtags)
reqs = parser.parse_all(issues2)

parser2 = UserTestsParser()
tests = parser2.parse_excel("C:\\Users\\Nathan\\Documents\\Dropbox\\FRS\\FRS Tests 190226.xlsx")

parser3 = WebFRSIssuetrackerParser(rtags)
reqs2 = parser3.parse_all(issues2)

allitems = reqs + tests + reqs2

def missing_parents(req, ref, reqs):
    if req.type == "TEST":
        return 'ignore'
    else:
        return 'fix'

rex = RequirementExtracter(rtags, ttags)
rex.config.missing_parents = missing_parents
rows = rex.extract(allitems)

In [12]:
import networkx as nx
import itertools

def impacted_nodes(G, node):
    a = nx.shortest_path_length(G, target=node)
    d = nx.shortest_path_length(G, source=node)
    return {n for n in itertools.chain(a, d)}

def impact_graph(G, node):
    r = impacted_nodes(G, node)
    return G.subgraph(r)

def items_to_graph(items, reqs):
    G = nx.DiGraph()
    for req in items:
        if not req.obs:
            for ref in req.refs:
                if ref in reqs:
                    G.add_edge(ref, req.tag)
    return G

def dictify(items):
    out = {}
    for i in items:
        if not i.obs:
            if i.tag in out:
                print("Warning: duplicate for tag: %s"%i.tag)
            out[i.tag] = i
    return out

def add_implicit_local_refs(items):
    for req in items:
        hpref = req.tag.rsplit(".", 1)[0]
        if hpref != req.tag:
            req.refs.add(hpref)
            
def make_impact_graph(G, node, file):
    SG = impact_graph(G, node)
    nx.write_graphml(G, file)

In [13]:
_imkey = re.compile(r"(%s)(\d+)\.?([\d\.]*)" % "|".join(['URS', 'FRS', 'SDS'])).match

def key_match(key):
    m = _imkey(key)
    if m:
        type, first, others = m.groups()
        return type+first, others
    return key, ""

def root_for_type(items, typ):
    root = FRS.Root(key_match)
    for req in items:
        if req.type == typ and not req.obs:
            root.add(req.tag, 0)
    return root

In [16]:
def condensed_implement_graph(G, node, reqs):
    d = nx.shortest_path_length(G, source=node)
    return condensed_graph(d, reqs)

def combined_implement_graph(G, nodes, reqs):
    d = set()
    for n in nodes:
        try:
            nrefs = impacted_nodes(G, n)
        except nx.NodeNotFound:
            pass
        else:
            d.update(nrefs)
    return condensed_graph(d, reqs)

In [17]:
def copysubgraph(G, nodes):
    G2 = nx.DiGraph()
    G2.add_nodes_from(nodes)
    for s,t in G.edges():
        if s in G2 and t in G2:
            G2.add_edge(s,t)
    return G2

def combined_implement_graph3(G, new):
    d = set()
    for n in new:
        try:
            nrefs = impacted_nodes(G, n)
        except nx.NodeNotFound:
            pass
        else:
            d.update(nrefs)
    return combined_implement_graph2(G, d)

def combined_implement_graph2(G, new):
    #G2 = copysubgraph(G, new)
    return condensed_graph2(G2)
    
def condensed_graph2(G):
    return nx.algorithms.dag.transitive_reduction(G)

In [27]:
import py2cytoscape
from py2cytoscape.data.cyrest_client import CyRestClient

cy = CyRestClient()

In [28]:
def remove_interface_refs(G):
    interface = "4370", "4374"
    
    edges = list(G.edges())
    for a,b in edges:
        an = a[3:7]
        bn = b[3:7]
        if (an in interface) != (bn in interface):
            G.remove_edge(a,b)

In [132]:
def remove_diffnum_refs(G):
    edges = list(G.edges())
    G2 = nx.DiGraph()
    def g(r): return r[3:7]
    for a, b in edges:
        if g(a) != g(b):
            G2.add_edge(a,b)
    for n in list(G2.nodes()):
        p = n.rsplit(".", 1)[0]
        if p in G2 and n != p:
            G2.add_edge(p, n)
    return G2

def type_only(G, typ):
    G2 = nx.DiGraph()
    for a,b in list(G.edges()):
        if a.startswith(typ) and b.startswith(typ):
            G2.add_edge(a,b)
    return G2

In [154]:
import random
def rand_color():
    b16 = "0123456789ABCDEF"
    def c(): return random.randint(127, 255)
    return "#%02x%02x%02x"%(c(), c(), c())
    return "#" + "".join(b16[random.randint(0, 15)] for _ in range(6))

In [160]:
   
apn = allitems
apn = [n for n in apn if n.type != "TEST"]
add_implicit_local_refs(apn)
reqs = dictify(apn)

G = items_to_graph(apn, reqs)
#remove_interface_refs(G)
G2 = type_only(G, "URS")
remove_interface_refs(G2)
G3 = condensed_graph2(G2)

cy.session.delete()

data = py2cytoscape.util.from_networkx(G2)
mappings = {}
for node in data['elements']['nodes']:
    node = node['data']
    req = reqs[node['id']]
    node['type'] = req.type
    node['group'] = req.num.split('.',1)[0]
    mappings[node['group']] = mappings.get(node['group'], rand_color())
    node['new'] = reqs[node['id']].milestone == '3.0.8'
    
nw = cy.network.create(None, name="test2", collection="Test2", data=data)
cy.layout.apply("force-directed", nw)
style = cy.style.create("pbs_reqs", cy.style.get("default"))
style.create_discrete_mapping("group", "String", "NODE_FILL_COLOR", mappings)
cy.style.apply(style, nw)
#cy.style.apply(style, nw)

In [159]:
cy.style.apply(style, nw)

In [128]:
nx.algorithms.dag.is_directed_acyclic_graph??

In [62]:
style = cy.style.create("test", cy.style.get("default"))

In [115]:
style.create_discrete_mapping("group", "String", "NODE_FILL_COLOR", mappings)

In [143]:
class MyStyle(py2cytoscape.data.style.Style):
    def _Style__call_create_mapping(self, mapping):
        url = self._Style__url + 'mappings'
        r = requests.post(url, data=json.dumps([mapping]), headers=py2cytoscape.data.HEADERS)
        r.raise_for_status()
style = MyStyle('pbs_reqs')
style.create_discrete_mapping("group", "String", "NODE_FILL_COLOR", mappings)

HTTPError: 404 Client Error: Not Found for url: http://localhost:1234/v1/styles/pbs_reqs/mappings

In [87]:
py2cytoscape.data.style.Style??

In [72]:
mappings = {
    '4370': '#F21519'
}
style.create_discrete_mapping("group", 'String', "NODE_FILL_COLOR", mappings)

In [65]:
style.create_discrete_mapping??

In [75]:
cy.style.apply(style, nw)

In [71]:
style.get_mapping("NODE_FILL_COLOR")

mapping = {'map': [{'key': 'FRS', 'value': '#FFFF00'}],
 'mappingColumn': 'type',
 'mappingColumnType': 'String',
 'mappingType': 'discrete',
 'visualProperty': 'NODE_FILL_COLOR'}

r =requests.post("http://localhost:1234/v1/styles/test/mappings", data=json.dumps([mapping]), headers={'Content-Type':'application/json'})
r.status_code

201

In [56]:
import requests, json
c = requests.get("http://localhost:1234/v1/styles/visualproperties").content
d = json.loads(c.decode())
for vp in d:
    k = vp['visualProperty']
    v = vp.get('default', '')
    if k.startswith("NODE") and v.startswith("#"):
        print(vp['visualProperty'], vp.get('default', ""))

NODE_SELECTED_PAINT #FFFF00
NODE_PAINT #787878
NODE_LABEL_COLOR #000000
NODE_BORDER_PAINT #000000
NODE_FILL_COLOR #C80000


In [51]:
c[2]

34

In [61]:
cy.style.get("default")

{'defaults': [{'value': 10.0, 'visualProperty': 'COMPOUND_NODE_PADDING'},
  {'value': 'ROUND_RECTANGLE', 'visualProperty': 'COMPOUND_NODE_SHAPE'},
  {'value': 'org.cytoscape.view.presentation.property.NullVisualProperty$NullDataTypeImpl@3956b6ba',
   'visualProperty': 'DING_RENDERING_ENGINE_ROOT'},
  {'value': 'DefaultVisualizableVisualProperty(id=EDGE, name=Edge Visual Property)',
   'visualProperty': 'EDGE'},
  {'value': '', 'visualProperty': 'EDGE_BEND'},
  {'value': True, 'visualProperty': 'EDGE_CURVED'},
  {'value': '', 'visualProperty': 'EDGE_LABEL'},
  {'value': '#000000', 'visualProperty': 'EDGE_LABEL_COLOR'},
  {'value': 'Dialog.plain,plain,10', 'visualProperty': 'EDGE_LABEL_FONT_FACE'},
  {'value': 10, 'visualProperty': 'EDGE_LABEL_FONT_SIZE'},
  {'value': 255, 'visualProperty': 'EDGE_LABEL_TRANSPARENCY'},
  {'value': 200.0, 'visualProperty': 'EDGE_LABEL_WIDTH'},
  {'value': 'SOLID', 'visualProperty': 'EDGE_LINE_TYPE'},
  {'value': '#323232', 'visualProperty': 'EDGE_PAINT'},
